In [138]:
#讀入斷詞後的內容，讀入詞彙表，建立向量空間模型，並計算相似文件
#Big Data & Business Analytics, National Taiwan University

In [18]:
import csv

content=list() #用串列預備儲存所有內容

f = open('limit_up_ok.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows: 
  content.append(row[0])
f.close()
print(len(content))

f = open('limit_down_ok.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  content.append(row[0])
f.close()
print(len(content))

4194
7393


In [2]:
print(len(content)) #檢查總共有幾份內容

7393


In [4]:
# 這段程式碼的目的是：將兩個 CSV 檔案中出現的所有「不重複的詞彙」進行整合與編號。簡單說，就是幫所有詞建立一個「唯一詞彙表」，每個詞對應一個獨立編號。
termset=dict() #用字典預備儲存所有詞彙
no=0 #幫詞彙編號

f = open('limit_up_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

f = open('limit_down_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()
# 最後會得到一個不重複單詞的字典
# 比較好的做法應該是重複的詞要丟掉！-->自己去改

In [4]:
print(len(termset)) #檢查總共有幾個詞彙

108


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=termset,use_idf=True) #用給定的termset建構向量，並使用idf加權
print("done")

done


In [6]:
X = vectorizer.fit_transform(content) #用給定的文件集做計算


/Users/andy/opt/anaconda3/envs/monpa-env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [7]:
from scipy import sparse
sparse.save_npz("limit_model.npz", X) #寫入整個稀疏矩陣

In [9]:
# 確認一下npz檔

Z = sparse.load_npz("limit_model.npz")
print(Z.shape)           # 印出矩陣大小（文章數 × 詞彙數）
print(Z[0].toarray())    # 看第一篇文章的 TF-IDF 向量

NameError: name 'sparse' is not defined

In [1]:
#給定任何一段內容當作查詢
query='台股加權指數在最近9個交易日，從最高到最低點，跌了2,544點，創下史上最快速的失速列車紀錄；\
  12日台股盤中急挫1,418點，市場衰鴻遍野，據統計，盤中最多曾有711檔個股觸及跌停、占上市櫃的四成比重，\
  最後仍有251檔收跌停，其中，陽明等15檔股價亮燈跌停，仍有7千張以上賣單高掛，貨櫃三雄均入榜。'

#以下套用斷詞小範例
import monpa
from monpa import utils
str='' #暫存本篇斷詞後的內容用
sentence_list = utils.short_sentence(query) #斷句
for item in sentence_list:
  result_cut = monpa.cut(item) #斷詞
  for term in result_cut:
    term=term.strip() #去除前後多餘空白
    if(len(term)>1): #若詞長>1
      str=str+' '+term
print(str)

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.
 台股 加權 指數 最近 交易日 低點 2,544 創下 快速 失速 列車 紀錄 12 台股 急挫 1,418 市場 衰鴻 遍野 統計 最多 711 個股 觸及 跌停 市櫃 四成 比重 最後 251 跌停 其中 陽明 15 股價 亮燈 跌停 7千 以上 賣單 高掛 貨櫃 入榜


In [10]:
q=vectorizer.fit_transform([str]) #將查詢也套入同一個向量空間
print(q[0:1]) #印出該向量做觀察
# GPT說不要用fit

  (0, 51)	0.35355339059327373
  (0, 8)	0.35355339059327373
  (0, 5)	0.35355339059327373
  (0, 2)	0.35355339059327373
  (0, 1)	0.7071067811865475


In [17]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(q[0:1], X).flatten() #與給定文件集的向量做相似度計算
print(cosine_similarities) #印出與每一篇的相似度做觀察
print(len(cosine_similarities))

[0.26382271 0.55751494 0.5893735  ... 0.21122887 0.08344608 0.02462297]
7393


In [12]:
related_docs_indices = cosine_similarities.argsort() #將相似度由小至大做排序，並轉換成文件編號
print(related_docs_indices)

[ 900 4031 4032 ...  503 2293 5957]


In [13]:
d=related_docs_indices[:-6:-1] #從後面取5個文件編號 (也就是相似度最大的前5名)
print(d)

[5957 2293  503 3276 6619]


In [14]:
for i in d:
  print('doc ',i,' : '+content[i]) #印出這些文件內容

doc  5957  :  台積電 再度 填息 台股 早盤 攻高 中央社 記者 曾仁凱 台北 2024年 3月 18日 台積電 2330 今天 牽動 台股 指數 走勢 台積電 配發 股息 3.5 影響 加權 指數 30 不過 台積電 再度 上演 填息 台股 早盤 攻高 台積電 配發 股息 3.5 參考價 750 今天 股價 以上 開出 瞬間 完成 填息 股價 攻高 截至 上午 9時 10 台積電 股價 上漲 1.2 攻上 759 推升 台股 大漲 台積電 台股 權值 股價 漲跌 牽動 台股 指數 週五 台積電 棄權 賣壓 出籠 最後 爆出 3萬 大量 股價 急殺 重跌 31 造成 加權 指數 當天 重跌 255.4 19682.5 翻黑 終止 台積電 2019年 改採 每季 配息 過去 18 除息 順利 完成 填息 其中 13 除息 當天 快速 填息 填息 時間 長為 15 交易日
doc  2293  :  台積電 股價 不停 10 ETF 沾光 創新 AI 行情 延燒 台股 加權 指數 19386.92 作收 萬九 關卡 護國 神山 台積電 2330 衝上 738 創下 歷史 天價 挾帶 氣勢 國內 台股 ETF 18 股價 創新 其中 10 ETF
doc  503  :  台股 跌出 今年 第二 分析師 曝光 止跌 時間 台積電 定股神針 FTNN 新聞網 記者 蔡昀庭 台北 報導 台股 22 22256.99 作收 台股 第八 今年 第二 收盤 跌點 許多 好奇 台股 跌到 分析師 認為 台積電 重新 站上 960 台股 望止 台股 今日 以下 50.4 開低 指數 一路 崩跌 終場 加權 指數 下跌 612.27 收在 22256.99 跌幅 2.58 成交 金額 5500.73 許多 股民 憂心忡忡 解套 進場 深怕 韭菜 台股 重挫 許多 好奇 到底 何時 止跌 根據 Yahoo 財經 報導 顧德 投顧 分析師 張家豪 表示 觀察 加權 指數 6月 25日 頸線 22053 下跌 止穩 台積積 重新 站上 960 加權 指數 重新 站上 22053 7月 29日 窄基 指數 生效 台股 望止跌
doc  3276  :  台北 股市 台積電 跳空 900 台股 跌彈 200 時報 台北電 美股 觀望 台積電 ADR 收漲 0.24 台積電 2330 